In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import os
import time
import tensorflow as tf
import numpy as np
import sagemaker
sagemaker_session = sagemaker.Session()

In [8]:
#Number of categories to predict
LABEL_DIMENSIONS = 10
DATA_SHAPE = (28,28,1)
CHANN =1
# Train the dataset
BATCH_SIZE = 512
EPOCHS =60

In [4]:

def build_model(input_shape,number_dimensions):
    #using functional api
    input = tf.keras.Input(shape=input_shape)
    
    #Create convulational layer
    op = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3) , activation='relu')(input)
    op = tf.keras.layers.MaxPooling2D(pool_size=(2,2) , strides=2)(op) 
    
    op = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3) , activation='relu')(op)
    op = tf.keras.layers.MaxPooling2D(pool_size=(2,2) , strides=2)(op) 
    
    op = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3) , activation='relu')(op)
    
    op = tf.keras.layers.Flatten()(op)
    
    #Create dense layer
    op = tf.keras.layers.Dense(64,activation='relu')(op)
    
    predictions = tf.keras.layers.Dense(number_dimensions,activation='softmax')(op)
    
    model = tf.keras.Model(inputs=input , outputs=predictions)
    return model

In [5]:
def create_model():
    model = build_model(DATA_SHAPE,LABEL_DIMENSIONS)
    #Compile the model
    model.compile(loss = 'categorical_crossentropy',
             optimizer = tf.keras.optimizers.SGD(),
             metrics=['accuracy'])
    return model

In [6]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images = np.asarray(train_images,dtype=np.float32)/255
test_images = np.asarray(test_images,dtype=np.float32)/255

TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)
print (f"TRAINING_SIZE {TRAINING_SIZE} TEST_SIZE, {TEST_SIZE}.")

#Add channels 
train_images = train_images.reshape((TRAINING_SIZE,train_images.shape[1],train_images.shape[2],CHANN))
test_images = test_images.reshape((TEST_SIZE,test_images.shape[1],test_images.shape[2],CHANN))

# convert categorical values
train_labels = tf.keras.utils.to_categorical(train_labels,LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels,LABEL_DIMENSIONS)

#Cast labels to float
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)
print (f"TRAIN LABEL SIZE {train_labels.shape} TEST LABEL SIZE, {test_labels.shape}")


TRAINING_SIZE 60000 TEST_SIZE, 10000.
TRAIN LABEL SIZE (60000, 10) TEST LABEL SIZE, (10000, 10)


In [7]:
tf_model = create_model()

In [9]:
tf_model.fit(train_images,train_labels,batch_size=BATCH_SIZE, epochs=EPOCHS,shuffle=True)
tf_model.metrics_names

Train on 60000 samples
Epoch 1/60
60000/60000 [==============================] - 7s 119us/sample - loss: 2.2658 - accuracy: 0.2479
Epoch 2/60
60000/60000 [==============================] - 7s 111us/sample - loss: 1.9661 - accuracy: 0.4196
Epoch 3/60
60000/60000 [==============================] - 7s 122us/sample - loss: 1.1712 - accuracy: 0.5907
Epoch 4/60
60000/60000 [==============================] - 8s 129us/sample - loss: 0.9522 - accuracy: 0.6468
Epoch 5/60
60000/60000 [==============================] - 7s 110us/sample - loss: 0.8591 - accuracy: 0.6799
Epoch 6/60
60000/60000 [==============================] - 7s 110us/sample - loss: 0.7988 - accuracy: 0.7003
Epoch 7/60
60000/60000 [==============================] - 7s 111us/sample - loss: 0.7576 - accuracy: 0.7132
Epoch 8/60
60000/60000 [==============================] - 7s 110us/sample - loss: 0.7283 - accuracy: 0.7238
Epoch 9/60
60000/60000 [==============================] - 7s 111us/sample - loss: 0.6997 - accuracy: 0.7335
Epoch

['loss', 'accuracy']

In [10]:
#Prediction on sample data
predicttest_data = test_images[0:10]
predicttest_labels = test_labels[0:10]
result = tf_model.predict(predicttest_data)
for pred,exp in zip(result,predicttest_labels):
    print(f"Predicted:> {np.argmax(pred)} Actual:> {np.argmax(exp)}")

Predicted:> 9 Actual:> 9
Predicted:> 2 Actual:> 2
Predicted:> 1 Actual:> 1
Predicted:> 1 Actual:> 1
Predicted:> 6 Actual:> 6
Predicted:> 1 Actual:> 1
Predicted:> 4 Actual:> 4
Predicted:> 6 Actual:> 6
Predicted:> 5 Actual:> 5
Predicted:> 7 Actual:> 7


In [12]:
# Create SageMaker specific export
OUTDIR = 'keras_model'
import shutil
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf_model.save(OUTDIR)
print(tf_model)

INFO:tensorflow:Assets written to: keras_model/assets


In [13]:
loaded = tf.saved_model.load(OUTDIR)
print(list(loaded.signatures.keys()))  # ["serving_default"]

['serving_default']


In [14]:
loaded.tensorflow_version

'2.1.3'

In [15]:
#Version 1 added manually
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add(OUTDIR, arcname=f'{OUTDIR}/1', recursive=True)


In [16]:
inputs = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="model")

In [23]:
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_model = TensorFlowModel(
    model_data="s3://" + sagemaker_session.default_bucket() + "/model/model.tar.gz",
    role=role,
    framework_version="2.1.3"
)

In [24]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!CPU times: user 307 ms, sys: 28.2 ms, total: 335 ms
Wall time: 6min 31s


In [25]:
predictor.endpoint_name

'tensorflow-inference-2021-06-16-18-26-09-626'

In [ ]:
#from sagemaker.tensorflow.model import TensorFlowPredictor
#predictor = sagemaker.tensorflow.model.TensorFlowPredictor('tensorflow-inference-2021-06-11-21-28-10-024', sagemaker_session=sagemaker_session)

In [26]:
data = {"signature_name": "serving_default", "instances": predicttest_data.tolist()}
result =predictor.predict(data)
for pred,exp in zip(result['predictions'],predicttest_labels):
    print(f"Predicted:> {np.argmax(pred)} Actual:> {np.argmax(exp)}")

Predicted:> 9 Actual:> 9
Predicted:> 2 Actual:> 2
Predicted:> 1 Actual:> 1
Predicted:> 1 Actual:> 1
Predicted:> 6 Actual:> 6
Predicted:> 1 Actual:> 1
Predicted:> 4 Actual:> 4
Predicted:> 6 Actual:> 6
Predicted:> 5 Actual:> 5
Predicted:> 7 Actual:> 7


In [28]:
predictor.delete_endpoint()

ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-east-1:613438712599:endpoint-config/tensorflow-inference-2021-06-16-18-26-09-626".